A jupyter notebook to convert HMDB data into mgf files.

# Required Functions

In [7]:
def Construct_Properties_Json(path_in,path_out='Properties_data.json'):
    from rdkit import Chem
    """
    Parses HMDB provided structures.sdf file to retrieve structural data for HMDB metabolites. 

    Warning: Some structures in HMDB violate rdkit rules and cannot be correctly parsed. 
    These are flagged as the function proceeds and are returned in a list upon completion. 

    Required: 
    path_in(path): Location of the structures.sdf file from HMDB
    path_out(path): Location to output the output json file. 
    
    """
    import json
    structure_list = Chem.SDMolSupplier(path_in)
    Properties_Dict = {}
    Bad_Molecules = []
    for mol in structure_list:
        try: 
            HMDB_ID = mol.GetProp('HMDB_ID')
            try:
                SMILES_String = mol.GetProp('SMILES')
                INCHI_KEY = mol.GetProp('INCHI_KEY')
                GENERIC_NAME = mol.GetProp('GENERIC_NAME')
                Mol_Weight = mol.GetProp('MOLECULAR_WEIGHT')
                Properties_Dict[HMDB_ID]={'Generic_name':GENERIC_NAME,'Mol_Mass':Mol_Weight,'SMILES':SMILES_String,'HMDB_ID':HMDB_ID,'INCHI_KEY':INCHI_KEY}
            except:
                Bad_Molecules.append(HMDB_ID)
        except:
            pass
    with open(path_out, 'w') as fp:
        json.dump(Properties_Dict, fp)
    return Bad_Molecules

def Read_Properties_Json_to_Dict(path_in):
    """
    Read properties from saved JSON file into properties dict. 

    Required:
    path_in(path):Location of properties JSON.
    
    Returns: 
    Properties_Dict(dict): keys = HMDB ID, values = properties
    """
    import json
    with open(path_in, 'r') as fp:
        Properties_Dict = json.load(fp)
    return Properties_Dict
    
def get_peaks_from_xml(xml_path):
        """
        Retrieve msms peaks and metadata from XML file
        
        Required: 
        xml_path(path):Location of the spectrum xml file

        Output: 
        peaks(tupple list): peaks and intensitites
        charge (str): Assumed charge state based on ionization mode
        collision_energy(str): low, med, high collision energy used
        
        """
        tree = ET.parse(xml_path)
        root = tree.getroot() #ms-ms
        peaks = []
        ionization_mode = root.find('ionization-mode').text
        if ionization_mode == 'Positive' or ionization_mode== 'positive':
            charge = '1+'
        elif ionization_mode == 'Negative' or ionization_mode== 'negative':
            charge = '1-'
        elif ionization_mode == 'N/A' or ionization_mode == 'n/a': #This SHOULDN'T exist, but it does.. 
            charge = 'undefined'
        collision_energy = root.find('collision-energy-level').text
        for ms_ms_peak in root.findall('ms-ms-peaks/ms-ms-peak'):
            peak_charge = ms_ms_peak.find('mass-charge').text
            peak_intensity = ms_ms_peak.find('intensity').text
            peak_tupple = (peak_charge,peak_intensity)
            peaks.append(peak_tupple)
        return peaks,charge,collision_energy
        
def Write_mgf_from_DF(Spectra_Dataframe,generic_name,PEPMASS,charge,collision_energy,filename="my_mgf.mgf"):
    """
    Constructs an mgf file from a dataframe contianing spectral information. 

    Requires: 
    Spectra_Dataframe (dataframe): dataframe version of spectrum information.
    PEPMASS(float): The declaired PEPMASS to associate with the spectrum.  

    Output: 
    my_mgf.mgf(file output): mgf formatted file saved in current working directory. 
    
    
    """
    with open(filename,"w") as f: 
        f.write('BEGIN IONS\nPEPMASS='+str(PEPMASS)+'\nCHARGE='+charge+'\nCOLLISION_ENERGY='+collision_energy+"\nGENERIC_NAME="+generic_name+'\nNOTE= PEPMASS and charge assumed from HMDB mass and ionization data.\n')
        for i in Spectra_Dataframe.index.tolist():
            f.write(str(Spectra_Dataframe.loc[i]['m/z'])+"\t"+str(Spectra_Dataframe.loc[i]['Intensity'])+'\n')
        f.write('END IONS')
    f.close()

## Construct properties JSON from provided structures .sdf file

In [15]:
import os
structures_path_in = os.path.join('Input_Files','structures.sdf')
properties_path_out = os.path.join('Output_Files','Properties_Data.json')
Bad_Molecules = Construct_Properties_Json(structures_path_in,properties_path_out)


## Read in already constructed properties json

In [2]:
#provided in repository. 
properties_json_path = os.path.join('Output_Files','Properties_Data.json')
Properties_Dict = Read_Properties_Json_to_Dict(properties_json_path)
Properties_Dict['HMDB0000001']

{'Generic_name': '1-Methylhistidine',
 'Mol_Mass': '169.1811',
 'SMILES': 'CN1C=NC(C[C@H](N)C(O)=O)=C1',
 'HMDB_ID': 'HMDB0000001',
 'INCHI_KEY': 'BRMWTNUJHUMWMS-LURJTMIESA-N'}

# Construct MGF files from HMDB xml files

## Perform on all spectra

In [16]:
## Attempting to parse xml file 

import os
import xml.etree.ElementTree as ET
import pandas as pd
Output_Dir = os.path.join('Output_Files','mgf_files')

## for all xml files: 
badlist=[]
for spectrum_xml in os.listdir(os.path.join('Input_Files','hmdb_experimental_msms_spectra')):
    xml_path = os.path.join(os.path.join('Input_Files','hmdb_experimental_msms_spectra',spectrum_xml))
    HMDB_ID = spectrum_xml.split("_")[0]
    spectrum_number = spectrum_xml.rsplit("_")[-2]
    try:
        peaks,charge,collision_energy = get_peaks_from_xml(xml_path)
        if collision_energy == None: 
            collision_energy='N/A'
        peaks_DF = pd.DataFrame.from_records(peaks,columns=["m/z","Intensity"])
        PEPMASS = float(Properties_Dict[HMDB_ID]['Mol_Mass'])+1.0008 #assume pepmass because it is not listed.
        Generic_Name= Properties_Dict[HMDB_ID]['Generic_name']
        if charge == "1+":
            mode_folder = 'positive_mode'
        elif charge =="1-":
            mode_folder = 'negative_mode'
        elif charge == 'undefined':
            mode_folder = 'mode_not_defined'
        Write_mgf_from_DF(peaks_DF,Generic_Name,PEPMASS,charge,collision_energy,os.path.join(Output_Dir,mode_folder,HMDB_ID+"_spectrum_"+str(spectrum_number)+'.mgf'))
    except:
        badlist.append((HMDB_ID,spectrum_number))
# badlist can be called to view all spectra which could not be converted. Likely, this is due to a metadata mismatch. 


## Perform on specific spectra

In [ ]:
# for specific metabolites:
HMDB_ID = "HMDB0000001"
Filename_attempt = HMDB_ID+"_ms_ms_spectrum_1_experimental.xml"
xml_path = os.path.join('Input_Files','hmdb_experimental_msms_spectra',Filename_attempt)
spectrum_number = spectrum_xml.rsplit("_")[-2]
try:
    peaks,charge,collision_energy = get_peaks_from_xml(xml_path)
    if collision_energy == None: 
        collision_energy='N/A'
    peaks_DF = pd.DataFrame.from_records(peaks,columns=["m/z","Intensity"])
    PEPMASS = float(Properties_Dict[HMDB_ID]['Mol_Mass'])+1.0008 #assume pepmass because it is not listed. 
    Generic_Name= Properties_Dict[HMDB_ID]['Generic_name']
    if charge == "1+":
        mode_folder = 'positive_mode'
    elif charge =="1-":
        mode_folder = 'negative_mode'
    elif charge == 'undefined':
        mode_folder = 'mode_not_defined'
    Write_mgf_from_DF(peaks_DF,Generic_Name,PEPMASS,charge,collision_energy,os.path.join(Output_Dir,mode_folder,HMDB_ID+"_spectrum_"+str(spectrum_number)+'.mgf'))
except:
    print('Can not process this spectrum.')